In [1]:
print(__doc__)

from collections import namedtuple

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets

from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem.rdmolops import RDKFingerprint

Automatically created module for IPython interactive environment


In [2]:
xytuple = namedtuple('xytuple', ['x', 'y'])

def df_to_xy(df):
    ''' Processes a dataset and returns a tuple of the fingerprint and lipophilicity columns.
    
    Args:
        df: The dataset dataframe with molecular smiles and lipophilicity values.
    Returns:
        Named tuple of the fingerprint column as "x" and lipophilicity column as "y".
    '''
    # Make an np arr of lipophillicity values
    lip_arr = df['experimental logD7.4'].values
    # Make an np arr of fingerprints
    mol_series = df['SMILES structure'].apply(Chem.MolFromSmiles)
    RDK_fp_series = mol_series.apply(lambda mol: np.array(RDKFingerprint(mol, fpSize=4096)))
    RDK_fp_arr = np.matrix(RDK_fp_series.tolist())
    return xytuple(x=RDK_fp_arr, y=lip_arr)

In [4]:
processedtuple = namedtuple('processedtuple', ['X_train', 'y_train', 'X_test', 'y_test'])
def preprocess(filename="dataset.xlsx"):
    ''' Reads in dataset, performs preprocesing, and returns X_train, y_train, X_test, and y_test columns.
    
    Args:
        filename (optional): The filename to read the dataset from.
    Returns:
        Named tuple of X_train, y_train, X_test, and y_test.
    '''
    dataset_read = pd.read_excel(filename)
    dataset = dataset_read.iloc[:,0:4]
    training_set = dataset.loc[dataset['Tr=traning set\nTe=test set'] == "Tr"]
    test_set = dataset.loc[dataset['Tr=traning set\nTe=test set'] == "Te"]
    X_train, y_train = df_to_xy(training_set)
    X_test, y_test = df_to_xy(test_set)
    
    mm_scale = MinMaxScaler().fit(X_train)
    X_train_sc = mm_scale.transform(X_train)
    X_test_sc  = mm_scale.transform(X_test)
    n_scale = Normalizer().fit(X_train_sc)
    X_train_n = n_scale.transform(X_train_sc)
    X_test_n  = n_scale.transform(X_test_sc)
    return processedtuple(X_train=X_train_n, y_train=y_train, X_test=X_test_n, y_test=y_test)

In [5]:
from sklearn.metrics import mean_squared_error, r2_score
# Print formatting

class color:
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
def test_model(model, name, data):
    ''' Runs a sklearn model with provided data and prints out the results
    
    Args:
        model: sklearn model to use
        name: Name to use when results are printed
        data: dictionary of data to use with model with keys X_train, y_train, X_test, and y_test
    Returns:
        model
    '''
    print("\n{}{}{}{}\n".format(color.BOLD, color.UNDERLINE, name, color.END))
    model.fit(data["X_train"], data["y_train"])

    # predict on the unseen dataset
    y_pred_train = model.predict(data["X_train"])
    y_pred = model.predict(data["X_test"])
    mse = mean_squared_error(y_pred, data["y_test"])
    mse_train = mean_squared_error(y_pred_train, data["y_train"])
    rmse = np.sqrt(mse)
    r2 = model.score(data["X_test"], data["y_test"])

    # R-squared is a goodness-of-fit measubre for linear regression models, how much variance of the data vs our model
    # Explained variance score: 1 is perfect prediction
    print("{}r^2:{} {:.3f}".format(color.BOLD, color.END, r2))
    print('{}RMSE:{} {:.3f}'.format(color.BOLD, color.END, rmse))
    # We want a low mse
    print("{}MSE train:{} {:.3f}".format(color.BOLD, color.END, mse_train))
    print("{}MSE test:{} {:.3f}".format(color.BOLD, color.END, mse))
    
    print('So, in our model, {:.3f}% of the variability in Y can be explained using X. '.format(model.score(data['X_test'], data['y_test'])*100))
    print('Our model predicted most of the lipophilicity values in the test set within {:.2f} of the real value.'.format(rmse))

    ## Create a residual plot
    # Plot outputs training data
    plt.scatter(data["y_train"], y_pred_train, c='b',s=40, alpha=.5)
    # Plot outputs test data
    plt.scatter(data["y_test"],y_pred, c='g',s=40, alpha=.5)
    plt.xlabel("Actual lipo")
    plt.ylabel("Predicted lipo")
    plt.show()
    plt.scatter(model.predict(data["X_train"]), model.predict(data["X_train"])-data["y_train"], c='b',s=40, alpha=.5)
    plt.scatter(model.predict(data["X_test"]), model.predict(data["X_test"])-data["y_test"], c='g',s=40, alpha=.5)
    plt.title("Residual plot using training(blue) and test(green) data")
    plt.ylabel("Residuals")
    return model